In [2]:
# Static data
video_root = r'C:\Users\92860\OneDrive - Universiti Malaya\7015深度学习\Project\video'
label_path = r'C:\Users\92860\OneDrive - Universiti Malaya\7015深度学习\Project\video\label.txt'
inputs_root = r'C:\Users\92860\Desktop\inputs'
outputs_root = r'C:\Users\92860\Desktop\outputs'

In [2]:
# Generate all video image set for members to label
# work_root/video name_frame rate/o_1.jpg
import cv2
import os
import shutil
def read_and_save_clips(root,name,save_root):
    path = os.path.join(root,name)
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        print('Open failed',path)
        return
    #get frame rate
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    #get frame count
    frame_cnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    # height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    start_frame = 0
    end_frame = int(frame_cnt)
    # generate the save path and save each frame
    folder_path = os.path.join(save_root,name+'_'+str(frame_rate))
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    for f_idx in range(start_frame,end_frame):
        cap.set(cv2.CAP_PROP_POS_FRAMES, f_idx)
        ret,frame = cap.read()
        if not ret:
            print('read failed in frame',f_idx,name)
            continue
        img_path = os.path.join(folder_path,'o_'+str(f_idx)+'.jpg')
        # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cv2.imwrite(img_path,frame)

v_list = os.listdir(video_root)
for v_name in v_list:
    if v_name.endswith('.mp4'):
        read_and_save_clips(video_root,v_name,outputs_root)


KeyboardInterrupt: 

In [19]:
# Generate clips by label
# work_root/video_name/behavior_start frame_end frame/o_xxx.jpg
def read_and_save_clips(root,folder_name,behavior,start_frame,end_frame,save_root):
    folder_list = os.listdir(root)
    is_exist = False
    for f_n in folder_list:
        if f_n.startswith(folder_name):
            folder_name = f_n
            is_exist = True
            break
    if not is_exist:
        print('folder not found',folder_name)
    path = os.path.join(root,folder_name)
    frame_rate = float(folder_name.split('_')[-1])
    video_name = folder_name.split('_')[0]
    frame_name_list = os.listdir(path)
    outputs_dir = os.path.join(save_root,folder_name)
    if not os.path.exists(outputs_dir):
        os.makedirs(outputs_dir)
    for frame in range(start_frame,end_frame+1):
        frame_name = 'o_{}.jpg'.format(frame)
        frame_path = os.path.join(path,frame_name)
        if not os.path.exists(frame_path):
            print('fail to access frame', frame , video_name)
            continue
        outputs_clip_dir = os.path.join(outputs_dir,'{}_{}_{}'.format(behavior,start_frame,end_frame))
        if not os.path.exists(outputs_clip_dir):
            os.makedirs(outputs_clip_dir)
        outputs_frame_path = os.path.join(outputs_clip_dir,frame_name)
        shutil.copyfile(frame_path,outputs_frame_path)
        

with open(label_path) as f:
    f_list = f.readlines()
    
# f_list = f_list[-10:]
for label in f_list:
    print(label)
    label = '563 905'
    label = label.split(' ')[0:4] 
    label[2] = int(label[2])
    label[3] = int(label[3])
    read_and_save_clips(inputs_root,label[0]+'.mp4',label[1],label[2],label[3],outputs_root)


01 0 1100 1282
01 0 1975 2075
01 0 2938 3040
01 0 3321 3440
01 0 4069 4164
02 0 451 578
02 1 569 614
03 0 0 100
03 0 300 486
03 0 562 667
03 0 785 898
03 0 1001 1100
03 0 1202 1310
03 0 1336 1396
03 0 1656 1805
03 0 1944 1980
04 0 108 327
05 0 150 360
06 0 0 70
06 0 302 390
06 0 467 565
06 0 1229 1257
06 0 1748 1800
07 0 0 30
07 0 160 255
07 0 545 607
08 0 70 120
08 0 130 189
08 0 205 285
08 0 347 420
09 0 1 97
09 0 421 649
10 0 0 66
10 0 104 177
10 0 468 553
10 0 646 682
10 0 1225 1305
10 0 2077 2200
10 0 2693 2758
10 0 2967 3020
11 0 0 42
11 1 80 127
12 0 195 239
12 0 434 570
12 0 1043 1093
13 0 124 200
13 0 216 295
13 0 339 505
13 0 645 729
15 0 277 329
15 0 438 523
15 0 1145 1244
15 0 1686 1770
15 0 1783 1877
15 0 2138 2236
15 0 2276 2328
15 0 2412 2464
15 0 2512 2597
15 0 2959 3026
15 0 3063 3152
15 0 3168 3280
15 0 3389 3421
15 0 4710 4745
15 0 4760 4784
15 0 4804 4835
15 0 5253 5382
15 0 5768 5797
16 0 94 252
16 0 279 421
16 0 458 875
16 0 1256 1323
16 0 1434 1519
16 0 1889 2120

In [22]:
# Generate and save landmarks coordinates and images
# work_root/video_name/behavior_start frame_end frame/a_xxx.jpg
# work_root/video_name/behavior_start frame_end frame/xxx.txt
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)
  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]
    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

def save_landmarks_and_annoatated_image(origin_image_path,landmark_path,annotated_image_path):
    # STEP 2: Create an PoseLandmarker object.
    base_options = python.BaseOptions(model_asset_path='pose_landmarker_heavy.task')
    options = vision.PoseLandmarkerOptions(
        base_options=base_options,
        output_segmentation_masks=True,
        min_pose_detection_confidence= 0.2,
        min_pose_presence_confidence=0.2
    )
    detector = vision.PoseLandmarker.create_from_options(options)
    
    # STEP 3: Load the input image.
    image = mp.Image.create_from_file(origin_image_path)
    
    # STEP 4: Detect pose landmarks from the input image.
    detection_result = detector.detect(image)
    # print(detection_result.pose_landmarks)
    # STEP 5: Process the detection result. In this case, visualize it.
    annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
    cv2.imwrite(annotated_image_path,annotated_image)
    
    x_location = []
    y_location = []
    z_location = []
    pose_landmarks_list = detection_result.pose_landmarks
    for idx in range(len(pose_landmarks_list)):
        pose_landmarks = pose_landmarks_list[idx]
        for landmark in pose_landmarks:
            x_location.append(landmark.x)
            y_location.append(landmark.y)
            z_location.append(landmark.z)
    f = open(landmark_path , 'w')
    for xyz_list in [x_location,y_location,z_location]:
        for xyz in xyz_list:
            f.write(str(xyz))
            f.write(' ')
        f.write('\n')
    f.close()

import os
# get root list
v_list = os.listdir(outputs_root)
# loop each video folder
for v_name in v_list:
    # generate video folder path
    v_path = os.path.join(outputs_root,v_name)
    # get clips list under the video folder
    c_list = os.listdir(v_path)
    # loop each clips folder
    for c_name in c_list:
        # generate clip path
        c_path= os.path.join(v_path,c_name)
        # get file list
        f_list = os.listdir(c_path)
        for f_name in f_list:
            # find the original frame
            if f_name.startswith('o'):
                # generate file path
                f_path = os.path.join(c_path,f_name)
                # generate coordinates file path
                l_path = os.path.join(c_path,f_name.split('_')[-1].strip('.jpg') + '.txt')
                # generate annotated iamge file path
                a_path = os.path.join(c_path,''.join(['a_',f_name.split('_')[-1]]))
                # detect the landmarks and save the text and image.
                save_landmarks_and_annoatated_image(f_path,l_path,a_path)